In [2]:
import numpy as np #importa a biblioteca usada para trabalhar com vetores de matrizes
%run normalization_utils.py

X, Y, feature_names = build_bow()

Pegando notícias verdadeiras...
Pegando notícias falsas...
Quantidade de notícias verdadeiras: 3600
Quantidade de notícias falsas: 3600
Montando o BOW...
Quantidade de features: 44685
10 primeiros valores de Y: [0 0 0 0 0 0 0 0 0 0]
10 ultimos valores de Y: [1 1 1 1 1 1 1 1 1 1]


In [10]:
# Testando metodo para converter uma string numa amostra para predição
sentence = 'Tinha uma pedra no meio do caminho, no meio do caminho tinha uma pedra. Qual é o sentido da vida se a nossa existência é tão efemera.'

a = sentence_to_features(sentence, feature_names)

print(a[a!=0])
print(len(a))

[2 1 2 2 1 1 1]
44685


In [3]:
print('Calculando folds...')
folds = stratified_kfolds(Y, 5)
print('Folds calculados')

Calculando folds...
Folds calculados


In [4]:
Xtrain = X[folds[0][0]]
Ytrain = Y[folds[0][0]]

Xtest = X[folds[0][1]]
Ytest = Y[folds[0][1]]

In [17]:
%run Methods/Kneighbors.py
%run accuracy.py

bestAccuracy = 0

for i in range(1, , 2):
    Ypred = testSamples(Xtrain, Ytrain, Xtest, i)
    acc = accuracy(Ypred, Ytest)
    print('K: ', i, ' acc: ', acc)
    if bestAccuracy < acc:
        bestAccuracy = acc
        k = i
print('Best -> K: ', k, ' acc: ', bestAccuracy)

K:  1  acc:  0.7694444444444445
K:  3  acc:  0.7319444444444444
K:  5  acc:  0.7138888888888889
K:  7  acc:  0.7048611111111112
K:  9  acc:  0.7


In [ ]:
%run Methods/logisticRegression.py
%run accuracy.py

lambda_reg = 100

theta, Ypred = regression(Xtrain, Ytrain, Xtest, lambda_reg)
display(accuracy(Ypred, Ytest))

In [69]:
%run Methods/naiveBayes.py
%run accuracy.py

Ypred = naiveBayes(Xtrain, Ytrain, Xtest)
display(accuracy(Ypred, Ytest))

array([5.88668138e-04, 2.10238621e-05, 2.10238621e-05, ...,
       1.47167035e-04, 4.20477242e-05, 6.30715863e-05])

array([4.20477242e-05, 2.10238621e-05, 2.10238621e-05, ...,
       2.10238621e-05, 2.10238621e-05, 2.10238621e-05])

0.5

0.5

0.5055555555555555

In [ ]:
%run Methods/svmMethod.py
%run accuracy.py

kernel = 0
cost = 4
gamma = 1

model, Ypred = svmUse(Xtrain, Ytrain, Xtest, Ytest, kernel, cost, gamma)
acc = accuracy(Ypred, Ytest)
print('cost:', cost, ' kernel:', kernel, ' gamma:', gamma, 'accuracy:', acc)